In [ ]:
using LinearAlgebra
using ControlSystems
using ModelingToolkit
using DifferentialEquations
using NonlinearSolve
using Plots
using Interact
using Printf

# Exercise 1.1

Set up system differential equation with symbolic variables and parameters.

In [ ]:
@variables t x1(t) x2(t) y(t)
@parameters m1 m2 f1 f2 d1 d2 F(t)
D = Differential(t)

In [ ]:
eqs = [
    D(D(x1)) ~ 1 / m1 * (-f1 * (x1 - x2) - d1 * (D(x1) - D(x2)) + 2F),
    D(D(x2)) ~ 1 / m2 * (f1 * (x1 - x2) + d1 * (D(x1) - D(x2)) - f2 * x2 - d2 * D(x2)),
    y ~ x2
]
@named dynamics = ODESystem(eqs)

In [ ]:
# Symbolically transform to first order ODE system
fo_dynamics = ode_order_lowering(dynamics)

In [ ]:
# Remove equation for y(t) since it is an algebraic equation, not an ODE
simplified = structural_simplify(fo_dynamics)

In [ ]:
observed(simplified)

# Exercise 1.2

The state space representation can be obtained by differentiating system and output equation with respect to state and input signal.

In [ ]:
# Reorder equations to have state [x1, D(x1), x2, D(x2)] and output y in the system, declare parameter F as control
@named fo_sys = ODESystem(equations(fo_dynamics)[[3, 1, 4, 2, 5]], controls=[F])

# Get state space representation
A_sym = calculate_jacobian(fo_sys)[1:4, 1:4]

In [ ]:
b_sym = calculate_control_jacobian(fo_sys)[1:4, :]

In [ ]:
c_sym = calculate_jacobian(fo_sys)[5:5, 1:4]

In [ ]:
d_sym = calculate_control_jacobian(fo_sys)[5:5, :]

# Exercise 1.3

In [ ]:
# First, plug in provided values for symbolic parameters
A, b, c, d = map([A_sym, b_sym, c_sym, d_sym]) do M
    substitute.(M, ([m1 => 2, m2 => 1, f1 => 1, f2 => 1, d1 => 1, d2 => 1],)) .|> Symbolics.value .|> Float64
end
# Create state space system
sys = ss(A, b, c, d)

Check for controllability and observability:

In [ ]:
controllability_matrix = [b A*b A^2*b A^3*b]

In [ ]:
if det(controllability_matrix) != 0
    println("System is controllable.")
end

In [ ]:
observability_matrix = [c; c * A; c * A^2; c * A^3]

In [ ]:
if det(observability_matrix) != 0 
    println("System is observable.")
end

In [ ]:

plot(step(sys, 30), label="open-loop step response")

In [ ]:
plot(impulse(sys, 30), label="open-loop impulse response")

# Exercise 1.4

Feedback gain for state space controller:

Determine feedback vector r such that closed-loop matrix
    $A - b  r^T$
has desired eigenvalues

In [ ]:
# Create symbolic variables for feedback gain vector
@variables r[1:4]
# Symbolic array needs to be scalarized into array of symbolic variables
Ap = A - b * r' |> Symbolics.scalarize

In [ ]:
# First compute characteristic polynomial symbolically
@variables λ
cp_expr = det(λ * I - Ap)
cp = build_function(cp_expr, λ) |> eval
cp(λ)

In [ ]:
# Find feedback r such that resulting system has desired eigenvalues/poles of transfer function
desired_poles = [-1, -1, -1 + im, -1 - im]
cp_poles(λ) = prod(λ - p for p in desired_poles) |> real
cp_poles(λ)

In [ ]:
eqs = [cp(λ) ~ cp_poles(λ) for λ in -2:1:1]

In [ ]:
# solve symbolic LES for feedback gain
r_val = Symbolics.solve_for(eqs, r)

# Exercise 1.5

Compute prefilter for state space controller

In [ ]:
f = r_val
v = 1 / ((c - d * f') * ((b * f' - A) \ b) + d)[]

In [ ]:
# Closed loop control system:
sys_cl = ss(A - b * f', v * b, c, d)

In [ ]:
# Plot (scaled) plant step response until t=30
fig = plot(step(0.5 * sys, 30), lab="plant")
## Plot closed-loop step response
plot!(step(sys_cl, 30), lab="state space controller")

# Exercise 1.6

Design PID controller with Ziegler-Nichols method

First part: Determine critical amplification


In [ ]:
# System matrix of closed loop system with P-controller
@variables kₚ
Ap = A - kₚ * b * c

In [ ]:
# Symbolic characteristic polynomial
cp_symb = det(λ * I - Ap)
cp = build_function(cp_symb, λ) |> eval
# At critical amplification the system has a purely imaginary eigenvalue λ=im*ω
@variables ω
cp(im * ω)

In [ ]:
# Solve nonlinear equation cp(im * ω) = 0,
# i.e. find critical amplification kₚ and frequency ω where characteristic polynomial has zero at purely imaginary eigenvalue
eqs = cp(im * ω) ~ 0
@named nles = NonlinearSystem(eqs, [kₚ, ω], [])

In [ ]:
initial = [2.0, 2.0]
nl_prob = NonlinearProblem(nles, initial)
k_crit, omega = solve(nl_prob, NewtonRaphson())

# Exercise 1.7

Calculate periodic time of oscillation at critical amplification.

In [ ]:
f_crit = omega / 2π
T_crit = 1 / f_crit

# Alternatively: With an experiment

Increase gain until reaching stability boundary

In [ ]:
@manipulate for k = 0:0.02:2
    sys_p = ss(A - k * b * c, k * b, c, d)
    sol = step(sys_p, 20)
    # find local minima to determine periodic time
    i_min = findall(eachindex(sol.y)) do i
        x = sol.y
        return (1 < i < length(x)) && x[i-1] > x[i] < x[i+1]
    end
    # consider time difference between local minima
    T_arr = diff(sol.t[i_min])
    # average time between minima to get periodic time
    T = sum(T_arr) / length(T_arr)
    
    # plotting
    p = plot(step(sys_p, 20), lab="P controller")
    if length(i_min) >= 2
        i_min = i_min[1:2]
        t_min = sol.t[i_min]
        y_min = sol.y[i_min]
        plot!(t_min[[1,2,2]], y_min[[1,1,2]], lab="T")
    end
    hbox(vbox(
        @sprintf("T = %.2f", T), hskip(0em)
    ), hskip(5em), p)
end

Critical amplification is where system reached stability boundary. The periodic time can then be rear off.

# Exercise 1.8

Apply Ziegler-Nichols rules

In [ ]:
k_p = 0.6 * k_crit
T_I = 0.5 * T_crit
T_D = 0.12 * T_crit
k_p, T_I, T_D

Simulate resulting PID controller

In [ ]:
controller = pid(k_p, T_I, T_D)
openloop = tf(sys) * controller # multiply system and controller for serial combination

sys_pid = feedback(openloop)    # closed-loop system
plot!(fig, step(sys_pid, 30), lab="PID controller")
